# Overview
Calculate flight percentages that we can shifting inside ASEAN with these contexts  below:

- Domestic
- International
    - Among ASEAN countries
    - Inbound
    - Outbound
- Among Lv3 airports
- Outbound from Lv3 airports
- Among lv2+lv3 airports
- Outbound from Lv2+lv3 airports

In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

In [4]:
data_dir = "/home/shared/data/oag"
airport_data_path = "/home/danhle/AIATFM/data_preparation/deepflightplan/datasets/airports/iata-icao.csv"

# data_dir = Path(data_dir).glob("2024*.zip")
data_dir = Path(data_dir).glob("2024jan*.zip")
data_dir = sorted(data_dir)
# data_dir.remove(Path("/home/shared/data/oag/2024jan1_15_asean.zip"))
data_dir

[PosixPath('/home/shared/data/oag/2024jan1_15_asean.zip')]

In [6]:


# Full data columns
# Index(['Carrier Code', 'Dep Airport Code', 'Dep Airport Name', 'Dep City Name',
#        'Dep DOT Country Name', 'Dep Region Name', 'Arr City Name',
#        'Arr Airport Code', 'Arr Airport Name', 'Arr DOT Country Name',
#        'Arr Region Name', 'International/Domestic', 'Flying Time',
#        'Elapsed Time', 'Local Dep Time', 'Local Arr Time',
#        'Specific Aircraft Code', 'General Aircraft Code',
#        'Aircraft Range (km)', 'Seats', 'GCD (km)', 'GCD (NM)', 'Routing',
#        'Aircraft Cruise Speed (MPH)', 'Aircraft Hold Volume (M3)', 'Flight No',
#        'Carrier Name', 'FlyingTime (Minutes)', 'Economy seats (Total)',
#        'Business seats (Total)', 'Seats (Total)', 'Time series',
#        'from/to ASEAN'],
#       dtype='object')

df = pd.DataFrame()
# Loop through all files in the directory
for file in sorted(data_dir):
    if file.is_file():
        print(file)
        tmp = pd.read_csv(file, compression="zip", low_memory=False)
        df = pd.concat([df, tmp], ignore_index=True)


df = df.sort_values(by=['Time series'])
df.reset_index(drop=True, inplace=True)
df.drop_duplicates(inplace=True)
print(len(df))

/home/shared/data/oag/2024jan1_15_asean.zip
1413601


In [9]:
# Filter out necessary columns
df = df[['Dep Airport Code', 'Arr Airport Code', 'International/Domestic', 'Time series', 'Dep DOT Country Name', 'Arr DOT Country Name']]

In [10]:
iata_icao_df = pd.read_csv(airport_data_path, low_memory=False)

In [11]:
ASEAN_countries = ["Brunei Darussalam", "Cambodia", "Indonesia", \
                   "Lao People's Democratic Republic", "Malaysia",\
                    "Myanmar", "Philippines", "Singapore", "Thailand",\
                    "Timor-leste", "Viet Nam"]

level_3_airport_icao = ['VTBD', 'VTCC', 'VTCT', 'VTSP', 'VTBS', \
                        'VTBU', 'VTSS', 'VTUO', 'VTSE', 'VTUK', \
                        'VTSG', 'VTCL', 'VTUL', 'VTBL', 'VTCH', \
                        'VTCI', 'VTBK', 'VTUW', 'VTUQ', 'VTUN', \
                        'VTPN', 'VTPI', 'VTSF', 'VTSN', 'VTCN', \
                        'VTSC', 'VTSK', 'VTPB', 'VTPP', 'VTCP', \
                        'VTBP', 'VTPH', 'VTSR', 'VTUV', 'VTBW', \
                        'VTUI', 'VTSH', 'VTPO', 'VTSB', 'VTSM', \
                        'VTPT', 'VTPM', 'VTST', 'VTBO', 'VTUU', \
                        'VTUD', 'VTSY', 'VTUZ', 'WSSS', 'VDPP', \
                        'VDSA', 'VDSV']

level_2_airport_icao = ['WIII', 'WADD', 'WARR', 'WBGG', 'WBKK', \
                        'WMKI', 'WMKJ', 'WMKL', 'WMKP', 'WMSA', \
                        'WMKK', 'RPLL', 'RPLC', 'RPVM', 'RPSP', \
                        'VYYY', 'VYMD', 'VYNT', 'VVTS', 'VVNB', \
                        'VVDN', 'VVCI', 'VVCR', 'VVPQ', 'VVVD', \
                        'VVVH', 'VVPB', 'VVCT', 'VVDL']

level_3_airport_iata = iata_icao_df[iata_icao_df['icao'].isin(level_3_airport_icao)]['iata'].tolist()
level_2_airport_iata = iata_icao_df[iata_icao_df['icao'].isin(level_2_airport_icao)]['iata'].tolist()

In [29]:
# print("total airport icao",len(level_3_airport_icao) + len(level_2_airport_icao))
# print("total airport iata",len(level_3_airport_iata) + len(level_2_airport_iata))
print("total airport icao", len(set(level_3_airport_icao) | set(level_2_airport_icao)))
print("total airport iata", len(set(level_3_airport_iata) | set(level_2_airport_iata)))

total airport icao 81
total airport iata 72


In [12]:
def str_datetime_to_month(str_datetime):
    return str_datetime.split("-")[1]

def str_datetime_to_date(str_datetime):
    return str_datetime.split("-")[2]

In [13]:
# Assuming ASEAN_countries is a predefined list or set of country names
df['from/to ASEAN'] = df['Dep DOT Country Name'].isin(ASEAN_countries) | df['Arr DOT Country Name'].isin(ASEAN_countries)

In [14]:
asean_df = df[df['from/to ASEAN'] == True]
asean_df['Month'] = asean_df['Time series'].apply(str_datetime_to_month)
asean_df['Month'] = asean_df['Month'].astype(int)

asean_df['Date'] = asean_df['Time series'].apply(str_datetime_to_date)
asean_df['Date'] = asean_df['Date'].astype(int)

/tmp/ipykernel_820659/884023261.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asean_df['Month'] = asean_df['Time series'].apply(str_datetime_to_month)
/tmp/ipykernel_820659/884023261.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asean_df['Month'] = asean_df['Month'].astype(int)
/tmp/ipykernel_820659/884023261.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [15]:
print("Total flights: ", len(df))
print("ASEAN flights: ", len(df[df['from/to ASEAN'] == True]))
print("ASEAN percentage: ", len(df[df['from/to ASEAN'] == True]) / len(df) * 100, "%")

Total flights:  18519489
ASEAN flights:  1498466
ASEAN percentage:  8.091292367732176 %


## Domestic

In [16]:
asean_domestic_df = asean_df[asean_df['International/Domestic']== "Domestic"]

In [17]:
print("Domestic flights in ASEAN: ", len(asean_domestic_df))
print("Domestic flights in ASEAN percentage: ", len(asean_domestic_df) / len(asean_df) * 100, "%")

Domestic flights in ASEAN:  780715
Domestic flights in ASEAN percentage:  52.10094857007099 %


## International

In [18]:
asean_international_df = asean_df[asean_df['International/Domestic']== "International"]

In [19]:
print("International flights in ASEAN: ", len(asean_international_df))
print("International flights in ASEAN percentage: ", len(asean_international_df) / len(asean_df) * 100, "%")

International flights in ASEAN:  717751
International flights in ASEAN percentage:  47.89905142992901 %


In [20]:
among_asean_df = asean_international_df[asean_international_df['Dep DOT Country Name'].isin(ASEAN_countries) & asean_international_df['Arr DOT Country Name'].isin(ASEAN_countries)]
print("Among ASEAN flights: ", len(among_asean_df))
print("Among ASEAN flights percentage: ", len(among_asean_df) / len(asean_international_df) * 100, "%")

Among ASEAN flights:  200142
Among ASEAN flights percentage:  27.88460064841428 %


In [21]:
outbound_asean_df = asean_international_df[asean_international_df['Dep DOT Country Name'].isin(ASEAN_countries) & ~asean_international_df['Arr DOT Country Name'].isin(ASEAN_countries)]
print("Outbound flights in ASEAN: ", len(outbound_asean_df))
print("Outbound flights in International ASEAN percentage: ", len(outbound_asean_df) / len(asean_international_df) * 100, "%")
inbound_asean_df = asean_international_df[asean_international_df['Arr DOT Country Name'].isin(ASEAN_countries) & ~asean_international_df['Dep DOT Country Name'].isin(ASEAN_countries)]
print("Inbound flights in ASEAN: ", len(inbound_asean_df))
print("Inbound flights in International ASEAN percentage: ", len(inbound_asean_df) / len(asean_international_df) * 100, "%")

Outbound flights in ASEAN:  258733
Outbound flights in International ASEAN percentage:  36.04773800384813 %
Inbound flights in ASEAN:  258876
Inbound flights in International ASEAN percentage:  36.067661347737584 %


## Airport

In [22]:
# lv3_airport_df = asean_df[asean_df['Dep Airport Code'].isin(level_3_airport_iata)  asean_df['Arr Airport Code'].isin(level_3_airport_iata)]
# print("ASEAN flights with level 3 airports: ", len(lv3_airport_df))
# print("ASEAN flights with level 3 airports percentage: ", len(lv3_airport_df) / len(asean_df) * 100, "%")

among_lv3_airport_df = asean_df[asean_df['Dep Airport Code'].isin(level_3_airport_iata) & asean_df['Arr Airport Code'].isin(level_3_airport_iata)]
print("Among ASEAN flights with level 3 airports among ASEAN: ", len(among_lv3_airport_df))
print("Among ASEAN flights with level 3 airports among ASEAN percentage: ", len(among_lv3_airport_df) / len(asean_df) * 100, "%")

among_lv2_airport_df = asean_df[asean_df['Dep Airport Code'].isin(level_2_airport_iata) & asean_df['Arr Airport Code'].isin(level_2_airport_iata)]
print("Among ASEAN flights with level 2 airports among ASEAN: ", len(among_lv2_airport_df))
print("Among ASEAN flights with level 2 airports among ASEAN percentage: ", len(among_lv2_airport_df) / len(asean_df) * 100, "%")

Among ASEAN flights with level 3 airports among ASEAN:  133666
Among ASEAN flights with level 3 airports among ASEAN percentage:  8.920189046665056 %
Among ASEAN flights with level 2 airports among ASEAN:  214440
Among ASEAN flights with level 2 airports among ASEAN percentage:  14.31063500940295 %


In [23]:
not_lv2_and_lv3_airport_df = asean_df[~asean_df['Dep Airport Code'].isin(level_2_airport_iata) & ~asean_df['Arr Airport Code'].isin(level_2_airport_iata) & \
    ~asean_df['Dep Airport Code'].isin(level_3_airport_iata) & ~asean_df['Arr Airport Code'].isin(level_3_airport_iata)]
print("ASEAN flights with no level 2 and level 3 airports: ", len(not_lv2_and_lv3_airport_df))
print("ASEAN flights with no level 2 and level 3 airports percentage: ", len(not_lv2_and_lv3_airport_df) / len(asean_df) * 100, "%")

ASEAN flights with no level 2 and level 3 airports:  151703
ASEAN flights with no level 2 and level 3 airports percentage:  10.123886694793208 %


In [24]:
outbound_lv3_airport_df = outbound_asean_df[outbound_asean_df['Dep Airport Code'].isin(level_3_airport_iata) & ~outbound_asean_df['Arr Airport Code'].isin(level_2_airport_iata) & ~outbound_asean_df['Arr Airport Code'].isin(level_3_airport_iata)]
print("Outbound flights in ASEAN with level 3 airports: ", len(outbound_lv3_airport_df))
print("Outbound flights in ASEAN with level 3 airports percentage: ", len(outbound_lv3_airport_df) / len(asean_df) * 100, "%")

outbound_lv2_airport_df = outbound_asean_df[outbound_asean_df['Dep Airport Code'].isin(level_2_airport_iata) & ~outbound_asean_df['Arr Airport Code'].isin(level_2_airport_iata) & ~outbound_asean_df['Arr Airport Code'].isin(level_3_airport_iata)]
print("Outbound flights in ASEAN with level 2 airports: ", len(outbound_lv2_airport_df))
print("Outbound flights in ASEAN with level 2 airports percentage: ", len(outbound_lv2_airport_df) / len(asean_df) * 100, "%")

Outbound flights in ASEAN with level 3 airports:  126826
Outbound flights in ASEAN with level 3 airports percentage:  8.463722233270559 %
Outbound flights in ASEAN with level 2 airports:  125429
Outbound flights in ASEAN with level 2 airports percentage:  8.370493558078728 %


In [ ]:
among_lv2_and_lv3_airport_df = asean_df[ asean_df['Dep Airport Code'].isin(level_2_airport_iata) & asean_df['Arr Airport Code'].isin(level_3_airport_iata) | \
    asean_df['Dep Airport Code'].isin(level_3_airport_iata) & asean_df['Arr Airport Code'].isin(level_2_airport_iata) | \
    asean_df['Dep Airport Code'].isin(level_2_airport_iata) & asean_df['Arr Airport Code'].isin(level_2_airport_iata) | \
    asean_df['Dep Airport Code'].isin(level_3_airport_iata) & asean_df['Arr Airport Code'].isin(level_3_airport_iata)]
print("ASEAN flights with level 2 and level 3 airports: ", len(among_lv2_and_lv3_airport_df))
print("ASEAN flights with level 2 and level 3 airports percentage: ", len(among_lv2_and_lv3_airport_df) / len(asean_df) * 100, "%")

ASEAN flights with level 2 and level 3 airports:  454453
ASEAN flights with level 2 and level 3 airports percentage:  30.327881980638864 %


In [26]:
# outbound_lv2_and_lv3_airport_df = outbound_asean_df[(outbound_asean_df['Dep Airport Code'].isin(level_2_airport_iata) | outbound_asean_df['Arr Airport Code'].isin(level_3_airport_iata)) & \
#                                                     (~outbound_asean_df['Arr Airport Code'].isin(level_2_airport_iata) | ~outbound_asean_df['Dep Airport Code'].isin(level_3_airport_iata)) ]

outbound_lv2_and_lv3_airport_df = outbound_asean_df[(outbound_asean_df['Dep Airport Code'].isin(level_2_airport_iata) & ~outbound_asean_df['Arr Airport Code'].isin(level_2_airport_iata) & ~outbound_asean_df['Arr Airport Code'].isin(level_3_airport_iata)) | \
                                                    (outbound_asean_df['Dep Airport Code'].isin(level_3_airport_iata) & ~outbound_asean_df['Arr Airport Code'].isin(level_2_airport_iata) & ~outbound_asean_df['Arr Airport Code'].isin(level_3_airport_iata)) ]
                                                    
print("Outbound flights in ASEAN with level 2 and level 3 airports: ", len(outbound_lv2_and_lv3_airport_df))
print("Outbound flights in ASEAN with level 2 and level 3 airports percentage: ", len(outbound_lv2_and_lv3_airport_df) / len(asean_df) * 100, "%")

Outbound flights in ASEAN with level 2 and level 3 airports:  252255
Outbound flights in ASEAN with level 2 and level 3 airports percentage:  16.834215791349287 %


## High Demands

In [27]:
asean_df.columns

Index(['Dep Airport Code', 'Arr Airport Code', 'International/Domestic',
       'Time series', 'Dep DOT Country Name', 'Arr DOT Country Name',
       'from/to ASEAN', 'Month', 'Date'],
      dtype='object')

In [28]:
traffic_demand = asean_df.groupby('Month').agg(
                    total_flights=('Month', 'size'),  # Total number of flights per month
                ).reset_index()


# Calculate days per month (assuming 2024, a leap year)
# days_in_month = [31, 29, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]  # Jan to Dec for 2024
days_in_month = 15
traffic_demand['avg_traffic_per_day'] = traffic_demand['total_flights'] / days_in_month

# Calculate daily counts per month for mean and variance
daily_counts = asean_df.groupby(['Month', 'Date']).size().reset_index(name='daily_flights')
monthly_stats = daily_counts.groupby('Month').agg(
    # mean=('daily_flights', 'mean'),
    variance=('daily_flights', 'var')
).reset_index()

# Merge the statistics into the traffic_demand DataFrame
traffic_demand = traffic_demand.merge(monthly_stats, on='Month')


# Ensure all months (1-12) are present, filling missing ones with zeros
all_months = pd.DataFrame({'Month': range(1, 13)})
traffic_demand = all_months.merge(traffic_demand, on='Month', how='left').fillna({
    'total_flights': 0,
    'avg_traffic_per_day': 0,
    # 'mean': 0,
    'variance': 0
})

# Convert total_flights to integer
traffic_demand['total_flights'] = traffic_demand['total_flights'].astype(int)
traffic_demand['avg_traffic_per_day'] = traffic_demand['avg_traffic_per_day'].round(0).astype(int)
# traffic_demand['mean'] = traffic_demand['mean'].round(0).astype(int)
traffic_demand['variance'] = traffic_demand['variance'].round(0).astype(int)

print(traffic_demand)

    Month  total_flights  avg_traffic_per_day  variance
0       1         118976                 7932     29346
1       2              0                    0         0
2       3              0                    0         0
3       4              0                    0         0
4       5              0                    0         0
5       6              0                    0         0
6       7              0                    0         0
7       8              0                    0         0
8       9              0                    0         0
9      10              0                    0         0
10     11              0                    0         0
11     12              0                    0         0
